# Hacktiv8 Data Science Talent Fair : Cohort & RFM Analysis
Muhammad Fadlil Ismail
FTDS Batch 10

## 1. Objective
Tujuan dari pengerjaan tugas ini adalah:
1. Mengetahui produk dan kategori produk yang paling sering dibeli oleh pelanggan.
2. Mengetahui retention rate dari pelanggan menggunakan Cohort Analysis.
3. Melakukan RFM analysis untuk mengetahui distribusi RFM score dari tiap-tiap pelanggan.

## 2. Import Library
Library yang digunakan pada notebook ini adalah sebagai berikut:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import plotly.express as px
import plotly.graph_objects as go

## 3. Loading & Cleaning Data
Berikut adalah proses loading dataset menggunakan library pandas:

In [2]:
df = pd.read_csv('user-events.csv')

Lakukan pengecekan terhadap 5 data teratas dan 5 data terbawah:

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


In [4]:
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
885124,2021-02-28 23:55:01 UTC,view,953226,2144415927553229037,NaN,NaN,219.94,1515915625611023730,FRLqIttxKU
885125,2021-02-28 23:58:05 UTC,view,1715907,2144415927049912542,electronics.video.tv,starwind,80.03,1515915625611024014,g6WqPf50Ma
885126,2021-02-28 23:58:09 UTC,view,4170534,2144415939364389423,electronics.clocks,amazfit,64.92,1515915625611024020,xNIJBqZdkd
885127,2021-02-28 23:58:14 UTC,view,888273,2144415921932861531,electronics.telephone,NaN,10.16,1515915625611024030,9pCbKMIcSx
885128,2021-02-28 23:59:09 UTC,view,743182,2144415935631458761,construction.tools.soldering,kada,65.08,1515915625556087775,BejOXRngEW


Berikut adalah info singkat mengenai dataset, tipe data apa yang tersimpan di dalam tiap-tiap kolom dan jumlah data yang tersimpan untuk tiap-tiap kolom.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885129 entries, 0 to 885128
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     885129 non-null  object 
 1   event_type     885129 non-null  object 
 2   product_id     885129 non-null  int64  
 3   category_id    885129 non-null  int64  
 4   category_code  648910 non-null  object 
 5   brand          672765 non-null  object 
 6   price          885129 non-null  float64
 7   user_id        885129 non-null  int64  
 8   user_session   884964 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 60.8+ MB


Lakukan pengecekan missing value pada dataset.

In [6]:
df.isna().sum()

event_time            0
event_type            0
product_id            0
category_id           0
category_code    236219
brand            212364
price                 0
user_id               0
user_session        165
dtype: int64

Ternyata terdapat missing value pada kolom category_code dan brand. Selanjutnya lakukan manipulasi data dapa kolom category_code untuk mendapatkan kategori dan produk pada tiap-tiap baris dataset.

In [13]:
df['category'] = df.category_code.apply(lambda x: x.split('.')[0] if x is not np.nan else np.nan)
df['product'] = df.category_code.apply(lambda x: x.split('.')[-1] if x is not np.nan else np.nan)

Kolom event_time masih berformat string, ubah formatnya ke datetime dan hanya ambil tanggalnya saja.

In [14]:
df['event_time'] = pd.to_datetime(df['event_time']).dt.to_period('D')

c:\Users\Muhammad Fadlil\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


Simpan dataframe yang sudah di-edit untuk digunakan pada proses deployment :

In [51]:
df.to_csv('user-events2.csv')

Load ulang dataframe untuk memudahkan proses kedepannya:

In [209]:
df = pd.read_csv('user-events2.csv')
df['month'] = pd.to_datetime(df['event_time']).dt.month
df['event_time'] = pd.to_datetime(df['event_time'])
df['month'] = df['month'].map({9:1, 10:2, 11:3, 12:4, 1:5, 2:6})

## 4. Exploratory Data Analysis
Berikut adalah proses Exploratory Data Analysis, pertama kita akan menghitung terlebih dahulu nilai total_sales, total_quantity_sold dan total_active_customer selama 6 bulan berjalan sesuai tanggal pada dataset. Nilai ini nantinya akan digunakan untuk membuat scorecard pada dashbord.

Nilai total_sales (pendapatan kotor) selama 6 bulan adalah sebagai berikut:

In [47]:
total_sales = df[df.event_type=='purchase'].price.sum()
total_sales

5125395.62

Nilai total_quantity_sold (banyaknya jumlah barang terjual) adalah sebagai berikut:

In [48]:
total_quantity_sold = len(df[df.event_type=='purchase'])
total_quantity_sold

37346

active customer adalah pelanggan yang melakukan transaksi pembelian pada aplikasi, berikut adalah jumlah pelanggan aktif selama 6 bulan:

In [50]:
total_active_customer = df[df.event_type=='purchase'].user_id.nunique()
total_active_customer

21304

### a. Distribusi Aktivitas Pengunjung
Dari dataset kolom event_type diketahui bahwa tidak semua pelanggan yang berkunjung melakukan transaksi pembelian, ada yang hanya melihat dan hanya memasukkan barang ke keranjang, jumlah aktivitas view, cart dan purchase selama 6 bulan ditampilkan pada tabel di bawah ini:

In [71]:
buat_pie = df_eda["event_type"].value_counts().reset_index()
buat_pie

,index,event_type
0,view,793748
1,cart,54035
2,purchase,37346


Tabel tersebut ditampilkan dalam bentuk pie chart sebagai berikut:

In [72]:
fig = px.pie(buat_pie,values='event_type',names='index',title='Aktivitas Member',hole=.4,height=500,width=500)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

Diketahui bahwa sekitar 90% aktivitas adalah view atau melihat, sekitar 6% memasukkan barang ke keranjang dan sekitar 4% transaksi pembelian barang.

### b. Top 10 Kategori dan Produk yang dibeli pelanggan

Berikut adalah grafik top 10 kategori yang dibeli oleh pelanggan:

In [78]:
a = 'purchase'
b = 'category'
df_dist_event = pd.crosstab(df_eda[b], df_eda.event_type).sort_values('cart', ascending=False).head(10).reset_index()
fig = px.bar(df_dist_event,y=b,x=a,color=b,color_discrete_sequence=['indianred'],height=500,width=800, text_auto='.2s')
fig.update_layout(xaxis_title='Qty '+a,yaxis_title='Category',title='Top 10 '+b+' '+a, showlegend = False)
fig.show()

Dari grafik di atas terlihat bahwa kategori produk yang paling sering dibeli pelanggan adalah computers sebanyak sekitar 17000 pembelian, disusul electronics sebanyak sekitar 6800 pembelian dan stationery sebanyak sekitar 2900 pembelian selama 6 bulan berjalan.

In [79]:
a = 'purchase'
b = 'product'
df_dist_event = pd.crosstab(df_eda[b], df_eda.event_type).sort_values('cart', ascending=False).head(10).reset_index()
fig = px.bar(df_dist_event,y=b,x=a,color=b,color_discrete_sequence=['indianred'],height=500,width=800, text_auto='.2s')
fig.update_layout(xaxis_title='Qty '+a,yaxis_title='Category',title='Top 10 '+b+' '+a, showlegend = False)
fig.show()

Dari grafik di atas terlihat bahwa produk yang paling sering dibeli pelanggan adalah videocards sebanyak sekitar 6900 pembelian, disusul telephone sebanyak sekitar 4100 pembelian dan printer sebanyak sekitar 2600 pembelian selama 6 bulan berjalan.

Untuk top 10 kategori dan produk yang dilihat dan dimasukkan ke keranjang dapat dilihat pada dashboard.

### c. Aktivitas Pengunjung Perbulan
Berikut adalah grafik jumlah aktivitas pengunjung perbulannya selama 6 bulan untuk aktivitas view, cart dan purchase:

In [210]:
for aktivitas in ['view', 'cart', 'purchase']:
    lihat0 = df[df.event_type==aktivitas].groupby('month')[['product_id']].count().reset_index()
    fig2 = px.bar(lihat0,x='month',y='product_id',height=500,width=700, text_auto=True)
    fig2.update_traces(marker_color='#0000FF')
    fig2.update_layout(title = 'Jumlah '+ aktivitas+' per bulan',xaxis_title='Bulan', yaxis_title=' ')
    fig2.show()

Terlihat tren kenaikan jumlah aktivitas dari bulan pertama hingga bulan ke-enam untuk semua jenis aktivitas.

### d. Perbulan
Pada bagian ini akan dihitung distribusi pelanggan aktif, jumlah produk terjual, dan jumlah pendapatan kotor untuk tiap-tiap bulannya selama enam bulan.

Berikut adalah distribusi jumlah pelanggan aktif untuk tiap bulan, dimana pelanggan aktif adalah pelanggan yang melakukan transaksi pembelian :

In [116]:
lihat = df[df.event_type=='purchase'].groupby('month')[['user_id']].nunique().reset_index()
fig2 = px.bar(lihat,x='month',y='user_id',height=500,width=700, text_auto=True)
fig2.update_traces(marker_color='#5aa17f')
fig2.update_layout(xaxis_title='Bulan',title='Jumlah pelanggan aktif per bulan', yaxis_title=' ')
fig2.show()

Terlihat tren kenaikan jumlah pelanggan aktif dari bulan pertama hingga keenam.

Berikut adalah distribusi jumlah produk terjual untuk tiap bulan:

In [115]:
lihat2 = df[df.event_type=='purchase'].groupby('month')[['product_id']].count().reset_index()
fig2 = px.bar(lihat2,x='month',y='product_id',height=500,width=700, text_auto=True)
fig2.update_traces(marker_color='#5aa17f')
fig2.update_layout(xaxis_title='Bulan',title='Jumlah produk terjual per bulan', yaxis_title=' ')
fig2.show()

Terlihat tren kenaikan jumlah produk terjual dari bulan pertama hingga keenam.

Berikut adalah distribusi pendapatan kotor untuk tiap bulan:

In [117]:
lihat3 = df[df.event_type=='purchase'].groupby('month')[['price']].sum().reset_index()
fig2 = px.bar(lihat3,x='month',y='price',height=500,width=700, text_auto=True)
fig2.update_traces(marker_color='#5aa17f')
fig2.update_layout(xaxis_title='Bulan',yaxis_title='Jumlah Pendapatan Kotor ($)')
fig2.show()

Terlihat tren kenaikan pendapatan kotor dari bulan pertama hingga keenam.


## 5. Analisis Cohort dan RFM

### a. Cohort Analysis
Cohort Analysis adalah analisis terhadap retention rate dari pelanggan lama pada suatu bisnis. Retention rate sendiri memiliki arti seberapa banyak pelanggan lama yang bertahan sejak pertama kali mereka melakukan transaksi hingga sekarang. Pada notebook ini hanya akan dilakukan analisa terhadap pelanggan yang melakukan pembelian saja.

In [212]:
i = 'purchase'

Ambil kolom yang akan digunakan untuk analisa:

In [213]:
df_cohort = df[df.event_type==i][['user_id', 'event_time', 'price']]

kolom event_time berisi tanggal pembelian dari tiap-tiap transaksi, namun kita hanya memerlukan bulan pembelian saja, sehingga dibuat kolom baru yaitu  invoice_month yang berisi bulan dari tiap-tiap transaksi

In [214]:
def get_month(x) : return dt.datetime(x.year,x.month,1)
df_cohort['invoice_month'] = df_cohort['event_time'].apply(get_month)

Selanjutnya kita akan mencari bulan dimana tiap-tiap pelanggan pertama kali melakukan transaksi. Untuk melakukan hal ini kita perlu membuat dataframe baru yang berisi user_id dan bulan pertama user melakukan transaksi, kolom tersebut bernama cohort_month.

In [215]:
df_cohort2 = df_cohort.groupby('user_id').invoice_month.min().reset_index()
df_cohort2.columns = ['user_id', 'cohort_month']

Selanjutnya lakukan inner join kedua dataframe tersebut dengan key kolom user_id:

In [216]:
hasil = df_cohort.merge(df_cohort2, left_on = 'user_id', right_on = 'user_id')
hasil.head()

,user_id,event_time,price,invoice_month,cohort_month
0,1515915625519390468,2020-09-24,217.57,2020-09-01,2020-09-01
1,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09-01
2,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09-01
3,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09-01
4,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09-01


Lalu buat kolom baru yaitu month, yang berisi selisih antara bulan transaksi / invoice_month dengan bulan pertama pelanggan bertransaksi / cohort_month

In [217]:
hasil['month'] = (hasil['invoice_month'] - hasil['cohort_month'])
hasil['month'] = round(hasil['month'].dt.days/30) +1

In [218]:
hasil['cohort_month'] = hasil['cohort_month'].dt.to_period('M')
hasil.head()

,user_id,event_time,price,invoice_month,cohort_month,month
0,1515915625519390468,2020-09-24,217.57,2020-09-01,2020-09,1.0
1,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09,1.0
2,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09,1.0
3,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09,1.0
4,1515915625513574486,2020-09-24,123.35,2020-09-01,2020-09,1.0


Lakukan croisstab untuk kolom cohort_month dengan kolom month:

In [219]:
cross_tab = pd.crosstab(hasil.cohort_month, hasil.month)
cross_tab = cross_tab.divide(cross_tab.iloc[:, 0], axis=0).round(4)*100
cross_tab.reset_index(inplace=True, drop=True)

In [220]:
cross_tab.index = cross_tab.index.map({0:'Sep-20', 1:'Oct-20', 2:'Nov-20', 3:'Dec-20', 4:'Jan-21', 5:'Feb-21'})
cross_tab

month,1.0,2.0,3.0,4.0,5.0
Sep-20,100.0,3.87,0.79,0.20,0.79
Oct-20,100.0,2.10,0.71,0.06,0.11
Nov-20,100.0,1.51,0.59,0.16,0.00
Dec-20,100.0,1.68,0.46,0.00,0.00
Jan-21,100.0,3.07,0.00,0.00,0.00
Feb-21,100.0,0.00,0.00,0.00,0.00


Lalu lakukan visualisasi hasil crosstab:

In [221]:
fig = px.imshow(cross_tab, text_auto=True, color_continuous_scale='Blues',height=500,width=650)
fig.update_layout(yaxis_title='Bulan', xaxis_title='Cohort Index', title='Retention rates')
fig.show()

Cara membaca hasil cohort analysis:
Pada bulan sep-20 cohort index 1 terisi nilai 100, namun pada bulan sep-20 cohort index 2 terisi nilai 3.87, artinya dari seluruh pelanggan yang transaksi pertamanya bulan sep-20, hanya 3.87% saja yang melakukan transaksi di bulan berikutnya (Oct-20).

### b. RFM Analysis
RFM analysis adalah analisis terhadap pelanggan menggunakan 3 metrik, yaitu:
1. R (Recency) = selisih hari antara hari terakhir pelanggan berbelanja dengan tanggal analisis
2. F (Frequency) = jumlah transaksi yang dilakukan pelanggan dalam satu periode
3. M (Monetary) = jumlah uang yang dihabiskan pelanggan dalam satu periode

Pertama ambil kolom yang akan digunakan untuk analisa yaitu kolom user_id, event_time dan price:

In [30]:
df_purchase = df[df.event_type=='purchase'][['user_id', 'event_time', 'price']]

Selanjutnya tentukan analysis date, yaitu tangggal hari terakhir pada dataset + 1 hari

In [31]:
analysis_date = df_purchase.event_time.max() + dt.timedelta(days=1)
analysis_date

Period('2021-03-01', 'D')

Selanjutnya buat kolom invoice yang merupakan index dari dataframe, kolom ini nanti digunakan untuk menghitung jumlah frequency pembelian pelanggan.

In [32]:
df_purchase['invoice'] = df_purchase.index
df_purchase.head()

,user_id,event_time,price,invoice
45,1515915625519390468,2020-09-24,217.57,45
82,1515915625513574486,2020-09-24,123.35,82
100,1515915625518130982,2020-09-24,89.22,100
132,1515915625518130982,2020-09-24,89.22,132
150,1515915625519356010,2020-09-24,123.79,150


Selanjutnya hitung nilai Recency, Frequency dan Monetary dari tiap-tiap pelanggan menggunakan groupby user_id

In [33]:
rfm = df_purchase.groupby('user_id').agg({'event_time': lambda x : (analysis_date - x.max()),
                                      'invoice':'count','price': 'sum'}).reset_index()

In [34]:
rfm['event_time'] = rfm['event_time'].apply(lambda x: 1 if str(x)=='<Day>' else int(re.findall('[0-9]+', str(x))[0]))

In [35]:
rfm.columns = ['user_id', 'Recency', 'Frequency', 'Monetary']

Berikut adalah hasilnya:

In [222]:
rfm

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,1515915625353286099,149,1,119.03,1,1,3,5
1,1515915625353457259,153,1,55.16,1,1,2,4
2,1515915625353534622,146,3,57.15,1,3,2,6
3,1515915625353561691,96,2,345.72,2,2,4,8
4,1515915625353900095,137,2,57.85,1,2,2,5
...,...,...,...,...,...,...,...,...
21299,1515915625610995356,1,1,29.40,4,1,1,6
21300,1515915625610997879,1,3,881.65,4,3,4,11
21301,1515915625610999486,1,3,1444.62,4,3,4,11
21302,1515915625611008742,1,1,38.83,4,1,2,7


Selanjutnya hitung nilai R_score, F_score dan M_score (pada dataframe ini hanya ditulis R, F, M saja) berdasarkan quartil dari nilai Recency, Frequency dan Monetary yang sudah didapat. Nilai 4 artinya nilai yang paling baik dan 1 adalah nilai yang paling buruk. Lalu hitung nilai RFM_score dengan menjumlahkan nilai R_score, F_score dan M_score yang sudah didapat:

In [36]:
r_labels =range(4,0,-1)
m_labels=range(1,5)
rfm['R'] = pd.qcut(rfm['Recency'], q=4, labels = r_labels)
rfm['F'] = rfm.Frequency.apply(lambda x: 1 if x==1 else 2 if x==2 else 3 if x<=4 else 4)
rfm['M'] = pd.qcut(rfm['Monetary'],q=4,labels = m_labels)
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

Simpan dataframe yang sudah dibuat ke csv untuk keperluan deployment lalu load kembali untuk mencoba apakah file csv tersebut dapat berfungsi dengan baik:

In [194]:
rfm.to_csv('rfm.csv', index=False)

In [195]:
rfm2 = pd.read_csv('rfm.csv')
rfm2.head()

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,1515915625353286099,149,1,119.03,1,1,3,5
1,1515915625353457259,153,1,55.16,1,1,2,4
2,1515915625353534622,146,3,57.15,1,3,2,6
3,1515915625353561691,96,2,345.72,2,2,4,8
4,1515915625353900095,137,2,57.85,1,2,2,5


Hitung jumlah user untuk tiap-tiap nilai RFM_score:

In [207]:
b = rfm2.groupby('RFM_Score').user_id.count().reset_index()
b

,RFM_Score,user_id
0,3,1562
1,4,2419
2,5,3217
3,6,3679
4,7,3184
5,8,3015
6,9,2254
7,10,1196
8,11,569
9,12,209


In [208]:
fig2 = px.bar(b,x='RFM_Score',y='user_id',height=500,width=700, text_auto=True)
fig2.update_traces(marker_color='#5aa17f')
fig2.update_layout(xaxis_title='RFM_Score',title='Distribusi pengguna berdasarkan RFM score', yaxis_title='Jumlah pengguna')
fig2.show()

Paling banyak, user memiliki nilai RFM score sebesar 6, mayoritas RFM_score pelanggan berada dikisaran 5-8.

Selanjutnya hitung rata-rata Recency untuk tiap-tiap R_score :

In [201]:
a = rfm2.groupby('R').Recency.mean().reset_index()
a

,R,Recency
0,1,134.163650
1,2,94.327835
2,3,52.856870
3,4,18.133247


In [223]:
fig2 = px.bar(a,x='R',y='Recency',height=500,width=700, text_auto=True)
fig2.update_traces(marker_color='#5aa17f')
fig2.update_layout(xaxis_title='R score',title='Rata-rata recency untuk tiap-tiap R score', yaxis_title='Rata-rata recency (hari)')
fig2.show()

Terlihat bahwa pelanggan dengan R_score=1 memiliki rata-rata Recency sebesar 134 hari, pelanggan dengan R_score = 3 memiliki rata-rata recency 94 hari, pelanggan dengan R_score=3 memiliki rata-rata recency 53 hari dan R_score = 4 memiliki rata-rata recency 18 hari.

Untuk visualisasi rata-rata frequency untuk tiap-tiap F_score dan rata-rata Monetary untuk tiap-tiap M_score dapat dilihat pada dashboard.